In [1]:
import os
import sys
import urllib.request
import datetime
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
def get_request_url(url):
    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", "jL0_Mj8EcTrEmAvqNflt")
    req.add_header("X-Naver-Client-Secret", "XjFsAbKpCH")
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [3]:
def getNaverSearchResult(sNode, search_text, page_start, display):
    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % sNode
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(search_text), page_start, display)
    url = base + node + parameters
    
    retData = get_request_url(url)
    
    if (retData == None):
        return None
    else:
        return json.loads(retData)

In [4]:
def getPostData(post, jsonResult):
    
    title = post['title']
    description = post['description']
    link = post['link']
    
    #if 'blog.naver' in link :
    #    jsonResult.append({'title':title, 'description': description, 'link': link})
        
    return

In [5]:
def makeJson():

    jsonResult = []

    # 'news', 'blog', 'cafearticle', 'kin'

    sNode = 'blog'
    search_text = '일상'
    display_count = 100
    
    jsonSearch = getNaverSearchResult(sNode, search_text, 1, display_count)
    
    while ((jsonSearch != None) and (jsonSearch['display'] != 0)):
        for post in jsonSearch['items']:
            getPostData(post, jsonResult)
        
        nStart = jsonSearch['start'] + jsonSearch['display']
        jsonSearch = getNaverSearchResult(sNode, search_text, nStart, display_count)
    
    with open('%s_naver_%s.json' % (search_text, sNode), 'w', encoding='utf8') as outfile:
        retJson = json.dumps(jsonResult,
                        indent=4, sort_keys=True,
                        ensure_ascii=False)
        outfile.write(retJson)
        
    print ('%s_naver_%s.json SAVED' % (search_text, sNode))

In [6]:
def makeLinkList():
    items = json.load(open("일상_naver_blog.json", "r", encoding="utf-8"))
    links = []
    
    i = 1
    for item in items:
        print(str(i) + " : " + item["link"])
        links.append(item["link"])
            
        if i == 100:
            break
        i += 1
        
    
    return links

In [7]:
def textCrawling(links):
    
    wfile = open(os.getcwd() + "/일상_naver_blog.txt", mode='w', encoding='utf-8')
    
    idx = 1
    for link in links:
        driver = webdriver.Chrome("/Users/leesoyeon/chromedriver")
        driver.get(link)

        driver.switch_to.frame("mainFrame")
        html = driver.page_source
    
        bs = BeautifulSoup(html, 'html.parser')
        # print(bs.prettify())

        textResult_1 = bs.find_all('div', {'class':['se-text', '__se_component_area']})
        textResult_2 = bs.find_all('div', {'id':'postViewArea'})
        
        print("naver_blog_" + str(idx) + " ")
        
        wfile.write("naver_blog_" + str(idx))
        
        for text in textResult_1:
            print(text.get_text())
            wfile.write(text.get_text())
        
        for text in textResult_2:
            print(text.get_text())
            wfile.write(text.get_text())
        
        
        idx = idx + 1
        driver.close()
    
    wfile.close()

In [8]:
def main():
    makeJson()
    links = makeLinkList()
    #links = ["https://blog.naver.com/charmjouny/221272169614"]
    #result = textCrawling(links)
    
main()

[2019-11-08 19:28:44.614207] Url Request Success
[2019-11-08 19:28:45.228064] Url Request Success
[2019-11-08 19:28:45.843413] Url Request Success
[2019-11-08 19:28:46.456637] Url Request Success
[2019-11-08 19:28:47.070234] Url Request Success
[2019-11-08 19:28:47.687928] Url Request Success
[2019-11-08 19:28:48.299379] Url Request Success
[2019-11-08 19:28:48.914992] Url Request Success
[2019-11-08 19:28:49.530840] Url Request Success
[2019-11-08 19:28:50.143048] Url Request Success
HTTP Error 400: Bad Request
[2019-11-08 19:28:50.225314] Error for URL : https://openapi.naver.com/v1/search/blog.json?query=%EC%9D%BC%EC%83%81&start=1001&display=100
일상_naver_blog.json SAVED
